# 文件读取

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import faiss
import pandas as pd
import numpy as np
import torch

import os
os.chdir(r"C:\天池比赛")

In [2]:
df = pd.read_csv(r".\A_audio_recognition_dom_189_train_val_avg.txt", sep='\t')
#df_food = pd.read_csv(r'.\商品-清洗.txt')
df_query = df[df['dom']==1]
#df_query = df_query.head(10)

In [3]:
df_query

,uuid,text,dom
2,6968138b-204e-4a12-9366-89f004aa4f56,天猫精灵要个家常花菜炒肉片吧,1
3,50005371-9bfd-45fc-9232-2fd269bc7b5f,来碗家常紫菜蛋花汤吧,1
4,03a15e19-9092-4671-b66c-12990f7af090,天猫精灵要碗红枣山药养胃粥吧,1
5,6dc1df30-6d8c-40a5-9be9-52f398623f07,天猫精灵鲜鱼豆腐汤来一份吧,1
6,38b8fafe-19f0-4003-ab7f-642f5a78a1dd,天猫精灵少油少盐的就行,1
...,...,...,...
3991,9b2b32da-dba8-447d-b090-84b48acb823d,来碗番茄汤小浑炖面吧,1
3993,3136cbf3-fa99-4196-b70b-207f09a497e3,要个暖心莲藕猪肚汤,1
3996,1341f3d3-7933-4832-9079-8beb49832649,天猫精灵来点养胃的,1
3997,79ede6cc-ee69-4a21-89bb-7315a0b0f215,天猫精灵来碗热乎的小米南瓜粥吧,1


In [4]:
##模糊测试
#df_query = pd.read_excel(r".\模糊.xlsx")
#df_query = df_query[['uuid','dom','food_feature','feature_class']]

In [5]:
df_query

,uuid,text,dom
2,6968138b-204e-4a12-9366-89f004aa4f56,天猫精灵要个家常花菜炒肉片吧,1
3,50005371-9bfd-45fc-9232-2fd269bc7b5f,来碗家常紫菜蛋花汤吧,1
4,03a15e19-9092-4671-b66c-12990f7af090,天猫精灵要碗红枣山药养胃粥吧,1
5,6dc1df30-6d8c-40a5-9be9-52f398623f07,天猫精灵鲜鱼豆腐汤来一份吧,1
6,38b8fafe-19f0-4003-ab7f-642f5a78a1dd,天猫精灵少油少盐的就行,1
...,...,...,...
3991,9b2b32da-dba8-447d-b090-84b48acb823d,来碗番茄汤小浑炖面吧,1
3993,3136cbf3-fa99-4196-b70b-207f09a497e3,要个暖心莲藕猪肚汤,1
3996,1341f3d3-7933-4832-9079-8beb49832649,天猫精灵来点养胃的,1
3997,79ede6cc-ee69-4a21-89bb-7315a0b0f215,天猫精灵来碗热乎的小米南瓜粥吧,1


# 大模型提取关键信息

In [6]:
import os
from openai import OpenAI
#后续尝试批量请求加速时间
def food_feature(sentence):
    message = [
        {'role': 'system',
         'content': '''你是一个文本分析助手。输入为句子，输出为从句子中提取的菜名，如果没有具体菜名，则从句子中提取有关菜品的描述特征。
         例如：
        user:天猫精灵来碗红枣桂圆养胃粥要热乎的  assistant：红枣桂圆养胃粥  
        user:天猫精灵少油少盐的就行  assistant:少油少盐'''},
        {'role': 'user', 'content':sentence}]
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-606b64b8196d4f119a7fb1789c86ac7c", 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen2.5-32b-instruct", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=message,
        )
    return completion.choices[0].message.content

In [7]:
df_query['food_feature'] = df_query['text'].progress_apply(food_feature)

100%|██████████| 1996/1996 [39:06<00:00,  1.18s/it]  
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\8730497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['food_feature'] = df_query['text'].progress_apply(food_feature)


In [8]:
import os
from openai import OpenAI
#后续尝试批量请求加速时间
def food_class(sentence):
    message = [
        {'role': 'system', 
         'content': '''你要做一个二分类任务，如果输入为菜品，返回1，如果输入为对菜品的模糊描述，返回0。
        例如:
        user： 红枣桂圆养胃粥 assistant：1  
        user：软和的 assistant：0'''},
        {'role': 'user', 'content':sentence}]
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-606b64b8196d4f119a7fb1789c86ac7c", 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen2.5-32b-instruct", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=message,
        )
    return completion.choices[0].message.content

In [9]:
df_query['feature_class'] = df_query['food_feature'].progress_apply(food_class)

100%|██████████| 1996/1996 [35:14<00:00,  1.06s/it]  
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\2209501132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['feature_class'] = df_query['food_feature'].progress_apply(food_class)


# 召回

### bge——>embedding

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
path = "./BAAI"
# 初始化模型和分词器
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)
 


In [11]:
#转换词向量函数
def cls(text): 
    input = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model(**input).last_hidden_state
    cls_embedding = output[:, 0, :]
    cls_embedding = cls_embedding[0]
    return cls_embedding

In [12]:
#df_food = pd.read_csv(r'.\商品-清洗.txt')
#df_food['cls'] = df_food['item_name'].progress_apply(cls)

In [13]:
# 存储时将tensor转换为list
#df_food['cls'] = df_food['cls'].apply(lambda x: x.tolist())
#df_food.to_csv(r'./df_food.csv')

In [14]:


# 读取时将list转回tensor
import ast
df_food = pd.read_csv(r'./df_food.csv')
df_food['cls'] = df_food['cls'].apply(lambda x: torch.tensor(ast.literal_eval(x)))


In [15]:
df_query['cls'] = df_query['food_feature'].progress_apply(cls)

100%|██████████| 1996/1996 [04:23<00:00,  7.58it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\1331004436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['cls'] = df_query['food_feature'].progress_apply(cls)


### FAISS 存储

In [16]:
## https://blog.csdn.net/weixin_43114209/article/details/141716443

In [17]:
# 把向量堆叠成 numpy 数组（shape: N x dim）
embeddings = np.stack(df_food['cls'].values).astype('float32')
dim = embeddings.shape[1]

# 创建索引（使用 L2 距离）
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print(f"向量数量：{index.ntotal}")


向量数量：10990


In [18]:
def query(food_cls,feature_class):
    query_vector = food_cls.reshape(1, -1)
    D, I = index.search(query_vector, k=5)  # D: 距离, I: 索引
    formatted = list(df_food.loc[s,'item_name'] for i, s in enumerate(I[0]))
    if feature_class == '0':
        formatted += ['营养配餐粥','养胃小米粥','小份烂糊白菜','骨汤鸡蛋汤','清炒时蔬菜']
    return formatted

In [19]:
#查询
df_query['Query_faiss'] = df_query.progress_apply(lambda row: query(row['cls'],row['feature_class']),axis=1)

100%|██████████| 1996/1996 [00:07<00:00, 259.77it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\1287812245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['Query_faiss'] = df_query.progress_apply(lambda row: query(row['cls'],row['feature_class']),axis=1)


In [20]:
## nlp推荐
def nlp_recommend(food_name,food_choice):
    input = food_name+'+'+str(food_choice)
    message = [
        {'role': 'system', 'content': '''你是一个智能点餐助手。给你一个点餐文本，以及可能的菜品列表。你的任务是从菜品列表中选出最符合点餐文本的菜品，并且只返回该菜品名称。
                    例如：
                    user：天猫精灵要个家常花菜炒肉片吧+[花菜炒肉片, 菜花炒肉片, 家常炒花菜, 花菜家常炒, 花菜炒肉丝]
                    assistant:花菜炒肉片'''},
        {'role': 'user', 'content':input}]
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-606b64b8196d4f119a7fb1789c86ac7c", 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen2.5-32b-instruct", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=message,
        )
    return completion.choices[0].message.content

In [21]:
df_query['nlp_recommend'] = df_query.progress_apply(lambda row: nlp_recommend(row['text'], row['Query_faiss']), axis=1)

100%|██████████| 1996/1996 [42:41<00:00,  1.28s/it]  
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\862560796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_recommend'] = df_query.progress_apply(lambda row: nlp_recommend(row['text'], row['Query_faiss']), axis=1)


In [22]:
df_query

,uuid,text,dom,food_feature,feature_class,cls,Query_faiss,nlp_recommend
2,6968138b-204e-4a12-9366-89f004aa4f56,天猫精灵要个家常花菜炒肉片吧,1,家常花菜炒肉片,1,"[tensor(0.8875), tensor(0.1605), tensor(-0.554...","[花菜炒肉片, 菜花炒肉片, 家常炒花菜, 花菜家常炒, 花菜炒肉丝]",花菜炒肉片
3,50005371-9bfd-45fc-9232-2fd269bc7b5f,来碗家常紫菜蛋花汤吧,1,紫菜蛋花汤,1,"[tensor(0.7710), tensor(0.2389), tensor(-0.238...","[紫菜蛋花汤, 蛋花紫菜汤, 紫菜蛋花儿汤, 营养紫菜蛋花汤, 美味紫菜蛋花汤]",紫菜蛋花汤
4,03a15e19-9092-4671-b66c-12990f7af090,天猫精灵要碗红枣山药养胃粥吧,1,红枣山药养胃粥,1,"[tensor(1.6882), tensor(0.9229), tensor(0.5272...","[红枣山药粥, 山药红枣粥, 红枣养胃粥, 山药大麦红枣粥, 红枣山药大麦粥]",红枣山药粥
5,6dc1df30-6d8c-40a5-9be9-52f398623f07,天猫精灵鲜鱼豆腐汤来一份吧,1,鲜鱼豆腐汤,1,"[tensor(-0.1967), tensor(0.9198), tensor(0.711...","[三鲜豆腐汤, 鲫鱼豆腐汤, 豆腐鲫鱼汤, 水鱼豆腐汤, 海鲜豆腐汤]",鲫鱼豆腐汤
6,38b8fafe-19f0-4003-ab7f-642f5a78a1dd,天猫精灵少油少盐的就行,1,少油少盐,0,"[tensor(0.6344), tensor(0.7226), tensor(0.4432...","[三荤两素加米饭, 小米粥或者玉米粥, 一荤两素加米饭, 三荤二素一汤一饭, 三荤二素一饭一...",清炒时蔬菜
...,...,...,...,...,...,...,...,...
3991,9b2b32da-dba8-447d-b090-84b48acb823d,来碗番茄汤小浑炖面吧,1,番茄汤小浑炖面,1,"[tensor(0.5655), tensor(0.6166), tensor(0.8255...","[番茄浓汤面, 番茄蛋汤面, 番茄鸡蛋汤面, 浓汤番茄鸡蛋面, 番茄挂面汤]",番茄浓汤面
3993,3136cbf3-fa99-4196-b70b-207f09a497e3,要个暖心莲藕猪肚汤,1,暖心莲藕猪肚汤,1,"[tensor(0.8180), tensor(0.6121), tensor(1.1287...","[莲子猪肚汤, 莲子猪肚汤饭, 暖胃猪肚鸡汤, 莲子枸杞猪肚汤, 养胃猪肚汤]",暖胃猪肚鸡汤
3996,1341f3d3-7933-4832-9079-8beb49832649,天猫精灵来点养胃的,1,养胃的,0,"[tensor(0.9521), tensor(0.8913), tensor(0.6510...","[养胃丨小米粥, 养胃的现熬皮蛋瘦肉粥, 养胃燕麦粥, 小米养胃粥, 红枣养胃粥, 营养配餐...",养胃丨小米粥
3997,79ede6cc-ee69-4a21-89bb-7315a0b0f215,天猫精灵来碗热乎的小米南瓜粥吧,1,小米南瓜粥,1,"[tensor(1.4292), tensor(0.2816), tensor(0.0894...","[小米南瓜粥, 南瓜小米粥, 小米粥南瓜粥, 小米健康南瓜粥, 南瓜小米养生粥]",小米南瓜粥


In [23]:
#df_query[['text','Query_faiss','nlp_recommend']].to_excel('./微调/微调训练集.xlsx')

In [24]:
#df_query0 = pd.read_excel(r".\模糊.xlsx")
#diff = pd.merge(df_query0,df_query,on='uuid',how='left')
#diff = diff[diff['nlp_recommend_x']!=diff['nlp_recommend_y']]

In [25]:
#diff.to_excel('abc.xlsx')

In [26]:
def faiss_1(s):
    return s[0]  # 输出: 花菜炒肉片

In [27]:
df_query['faiss_1'] = df_query['Query_faiss'].progress_apply(faiss_1)

100%|██████████| 1996/1996 [00:00<?, ?it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\4097527485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['faiss_1'] = df_query['Query_faiss'].progress_apply(faiss_1)


## faiss审核确保输出

In [28]:
df_query['nlp_cls'] = df_query['nlp_recommend'].progress_apply(cls)

100%|██████████| 1996/1996 [02:07<00:00, 15.62it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\3903690941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_cls'] = df_query['nlp_recommend'].progress_apply(cls)


In [29]:
def query_post(food_cls):
#返回最相近的菜品
    query_vector = food_cls.reshape(1, -1)
    D, I = index.search(query_vector, k=1)  # D: 距离, I: 索引
    return df_food.loc[I[0][0],'item_name']

In [30]:
#查询
df_query['nlp_post'] = df_query['nlp_cls'].progress_apply(query_post)

100%|██████████| 1996/1996 [00:04<00:00, 482.02it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\3269939578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_post'] = df_query['nlp_cls'].progress_apply(query_post)


In [31]:
df_query['nlp_post_cls'] = df_query['nlp_post'].progress_apply(cls)

100%|██████████| 1996/1996 [02:30<00:00, 13.26it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\433197166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['nlp_post_cls'] = df_query['nlp_post'].progress_apply(cls)


In [32]:
# 最终推荐菜品与提取特征间的相似度
def similarity(cls1,cls2):
    similarity = torch.cosine_similarity(cls1, cls2, dim=0)
    return similarity.item()

In [33]:
df_query['similarity'] = df_query.progress_apply(lambda row: similarity(row['cls'], row['nlp_post_cls']), axis=1)

100%|██████████| 1996/1996 [00:00<00:00, 45592.99it/s]
C:\Users\guofengrun\AppData\Local\Temp\ipykernel_16264\2678904413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query['similarity'] = df_query.progress_apply(lambda row: similarity(row['cls'], row['nlp_post_cls']), axis=1)


In [34]:
df_query[df_query['feature_class']=='1']['similarity'].mean()

0.936296645713889

# 提交文件格式

In [35]:
df_result = pd.merge(df, df_query[['uuid','nlp_post']], how='left', on='uuid')

In [36]:
df_result.columns

Index(['uuid', 'text', 'dom', 'nlp_post'], dtype='object')

In [37]:

df_result.to_csv(r".\result\Results506.txt", 
          sep='\t', 
          header=None, 
          index=False, 
          na_rep='')

In [38]:
# 存储时将tensor转换为list
#df_food['cls'] = df_food['cls'].apply(lambda x: x.tolist())
#df_food.to_csv(r'./df_food.csv',index=False)

In [39]:

'''
# 读取时将list转回tensor
import ast
df_food = pd.read_csv(r'./df_food.csv')
df_food['cls'] = df_food['cls'].apply(lambda x: torch.tensor(ast.literal_eval(x)))
'''

"\n# 读取时将list转回tensor\nimport ast\ndf_food = pd.read_csv(r'./df_food.csv')\ndf_food['cls'] = df_food['cls'].apply(lambda x: torch.tensor(ast.literal_eval(x)))\n"

In [40]:
#df_query[df_query['similarity']<0.7][['text','Query_faiss','nlp_post','similarity']].to_excel('./微调/模糊菜品430.xlsx')

In [41]:
#df_query.to_csv(r'./微调/df_query.csv')